In [ ]:
# - install pytorch 
# - install pytorch geometric:  https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html

# install additional libraries 
!pip install ogb
!pip install pip install pytorch-lightning

!pip install seaborn 
!pip install grakel
!pip install karateclub
!conda install future -y

In [ ]:
# install similarity forest [Isolation forest with similarity matrix as input]
# !git clone https://github.com/sfczekalski/similarity_forest
# !cd similarity_forest
# !pip install .

In [ ]:
# Some useful settings
import warnings
warnings.filterwarnings('ignore')

import matplotlib
dpi = 160
format = 'pdf'
# configure figure
matplotlib.rcParams.update({'font.size': 13})
# generate some new figure. for other times this path should be ignored or set as ''
# newpath = 'icml'

# Dataset tables
| Dataset      | Type                 |   Label  |   Meaning   |   Confirm | 
| -----------  | -----------          | ---------| ----------- | --------- |
| NCI1         | X & Y, Molecular     |     0    | The component (non-cell growth inhibition assay) is not effective for anit-HIV | - |
|       -      |           -          |     1    | The component (non-cell growth inhibition assay) is effective for anit-HIV | - |
| DD           | X & Non-X, Protein   |     0    |  691 enzymes| YES|
|       -      |           -          |     1    |  487 non-enzymes|  YES|
| PROTEINS     | X & Non-X, Protein   |     0    |  enzymes|  YES|
|       -      |           -          |     1    |  non-enzymes| YES|
| IMDB-BINARY  | X & Y, Social Net    |     0    | collabration network. movie genre: action | - |
|       -      |           -          |     1    | collabration network. movie genre: Romance| - |
| Mutagenicity | X & Non-X, Molecular |     0    | 2401 mutagens | YES|
|       -      |  Semnatic Anomaly    |     1    | 1936 nonmutagens | YES|
| PTC_MR       | X & Non-X, Molecular |     0    |  non-carcinogenicity  | Too small |
|       -      |  Semnatic Anomaly    |     1    |  carcinogenicity  |  Too small |
| AIDS         | X & Non-X. Molecular |     0    | 400  active, molecules with activity against HIV or not | YES|
|       -      |   Semnatic Anomaly   |     1    | 1600  inactive, molecules with activity against HIV or not | YES|
|ENZYMES 0-1   | X & Y, Protein       |     0/1  | specific type of enzyme| Too small ? |
|ENZYMES 2-3   | X & Y, Protein       |     2/3  | specific type of enzyme| Too small ? |
|COLLAB  0-1   | X & Y, Social Net    |     0/1  | collabration network. High Energy Physics or Condensed Matter|-|
|COLLAB  1-2   | X & Y, Social Net    |     1/2  | collabration network. Condensed Matter or Astro Physics|-|


# Dataset statistics

In [ ]:
from loader import *
import networkx as nx
from torch_geometric.utils import to_networkx


def data_stats(data_name):
    # get raw dataset
    dataset = load_data(data_name, return_raw=True)
    # split dataset by classes
    splits = [[] for _ in range(dataset.num_classes)] 
    for data in dataset:
        splits[data.y.item()].append(data)
    # compute statistics by class
    print(f'-------------------------  {data_name} --------------------------')
    for i, split in enumerate(splits):
        num_graph = len(split)
        num_features = dataset.num_node_features
        num_nodes_all = [] 
        num_edges_all = []
        num_components = []
        assortativities = []
        diameters = []
        degrees = []
        for graph in split:
            num_nodes_all.append(graph.num_nodes)
            num_edges_all.append(graph.num_edges)
            # create networkx graph
            g = to_networkx(graph, to_undirected=True) 
            if graph.x is not None:
                nx.set_node_attributes(g, name='label', values={i: torch.argmax(x).item() for i, x in enumerate(graph.x)})
            else:
                nx.set_node_attributes(g, name='label', values={i: 0 for i in range(graph.num_nodes)})
            # compute properties of the graph
            num_components.append(nx.number_connected_components(g))
            assort = nx.attribute_assortativity_coefficient(g, 'label')
            if not np.isnan(assort):
                assortativities.append(assort)

            degree = dict(g.degree()).values()
            degree = sum(degree) / len(degree)
            degrees.append(degree)
            # get largest component
            # Gcc = sorted(nx.connected_components(g), key=len, reverse=True)
            # gaint_g = g.subgraph(Gcc[0])
            # diameters.append(nx.radius(gaint_g))

        ave_num_nodes = sum(num_nodes_all) / len(num_nodes_all)
        ave_num_edges = sum(num_edges_all) / len(num_edges_all)
        ave_num_components = sum(num_components) / len(num_components)
        ave_assortativity = sum(assortativities) / (len(assortativities) + 1e-6)
        # ave_diameter = sum(diameters) / len(diameters)
        ave_diameter = 0
        ave_degree = sum(degrees) / len(degrees)
        print("Class %d: num_graphs=%d, num_labels=%d, ave_num_nodes=%f, ave_num_edges=%f, ave_diameter=%f, ave_degree=%f, ave_assort=%f, ave_num_comp=%f"
              %(i, num_graph, num_features, ave_num_nodes, ave_num_edges, ave_diameter, ave_degree, ave_assortativity, ave_num_components))

# datasets =  ['DD', 'PROTEINS', 'NCI1', 'IMDB-BINARY', 'Mutagenicity', 'AIDS', 'ENZYMES', 'COLLAB']
datasets = ['ENZYMES', 'REDDIT-MULTI-5K']
for data_name in datasets:
    data_stats(data_name)

# Dataset visualization

In [ ]:
import random
from torch_geometric.utils import to_networkx
import networkx as nx
import matplotlib.pyplot as plt
def plot_networkx(graph, role_labels, node_size=10):
    cmap = plt.get_cmap('tab20')
    x_range = np.linspace(0, 1, len(np.unique(role_labels)))
    coloring = {u: cmap(x_range[i]) for i, u in enumerate(np.unique(role_labels))}
    node_color = [coloring[role_labels[i]] for i in range(len(role_labels))]
    nx.draw(graph, #pos=nx.layout.fruchterman_reingold_layout(graph),
                        node_color=node_color, cmap='hot', node_size=node_size)

def visualize(data_name, k, node_size):
    dataset = load_data(data_name, return_raw=True)
    # split samples to each class
    class_sep_samples = {c:[] for c in range(dataset.num_classes)}
    for sample in dataset:
        class_sep_samples[sample.y.item()].append(sample)
    # sample k samples from each class and visualize them use networkx
    plt.figure(figsize=(k*3, dataset.num_classes*3))
    for c in range(dataset.num_classes):
        k_samples = random.sample(class_sep_samples[c], k)
        for i, d in enumerate(k_samples):
            if d.x is None:
                label = np.ones(d.num_nodes)
            else:
                label = d.x.argmax(dim=-1).numpy()
            G = to_networkx(d, to_undirected=True, remove_self_loops=True)
            plt.subplot(dataset.num_classes, k, i+1+k*c)
            plot_networkx(G, label, node_size=node_size)
            # if i == 0:
            plt.axis("on")
            if i == 0:
                plt.ylabel(f'Class={c}')
    plt.suptitle(f'Dataset: {data_name}')
    plt.tight_layout()
    plt.savefig(f'data_visualization/{data_name}.{format}', format=format)
    #plt.savefig(os.path.join(result_dir,'roc_vs_iter-{}-{}-{}.'.format(data_name, kernel_name, detector_name)+format), format = format) 

In [ ]:
visualize('REDDIT-MULTI-5K', 8, node_size=20)

In [ ]:
visualize('DD', 8, node_size=20)

In [ ]:
visualize('PROTEINS', 8, node_size=50)

In [ ]:
visualize('NCI1', 8, node_size=50)

In [ ]:
visualize('Mutagenicity', 8, node_size=50)

In [ ]:
visualize('AIDS', 8, node_size=50)

In [ ]:
visualize('ENZYMES', 8, node_size=50)

In [ ]:
visualize('IMDB-BINARY', 8, node_size=50)

In [ ]:
visualize('COLLAB', 8, node_size=50)

# WL/PK Kernel Experiments 
### -Model

In [ ]:
from kernel import *

### -Visualization

In [ ]:
# target: a function with kernel matrix and ys as input
# for each node in majority class, calulate a radius to nearliest , median and farest 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE, MDS

# always class 0 normal class 1 anomaly
def sort_kms(kms, ys):
    order = np.argsort(ys)
    ys = ys[order]
    kms = [km[order,:][:,order] for km in kms] 
    return kms, ys

def downsampling_kms(kms, ys, down_rate=1):
    # this need to called after sort
    index_anomaly = np.where(ys==1)[0] # always downsample label 1 anomaly class
    sub_index_anomaly = np.random.choice(index_anomaly, int(len(index_anomaly)*down_rate), replace=False) 
    sub_index = np.concatenate([np.where(ys==0)[0], sub_index_anomaly])
    ys_sub = ys[sub_index]
    kms_sub = [km[sub_index,:][:,sub_index] for km in kms]  
    return kms_sub, ys_sub

def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    import matplotlib.pyplot as plt
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

def visualize_oneclass(kms, ys, gap=2):
    # sort ys and kms 
    kms_all, ys_all = sort_kms(kms, ys)
    sub_index = np.where(ys_all==0)[0]
    #print(sub_index)
    kms0 = [km[sub_index,:][:,sub_index] for i, km in enumerate(kms_all) if i%gap==0] 
    fig, axes = plt.subplots(1, len(kms0), figsize=(len(kms0)*3,3), constrained_layout=True)
    for ii, kernel_matrix in enumerate(kms0): 
        aa = axes[ii].matshow(kernel_matrix, cmap=plt.get_cmap('RdBu').reversed())
        if ii == len(kms0)-1: colorbar(aa)
        axes[ii].set_xlabel('Similarity matrix for class 0')
        axes[ii].set_title('iter={}'.format(ii*gap+1))
    plt.tight_layout()
    
    return fig, axes

def visualize_disperity(kms, ys, roc_func, k=20, gap=2, downsampling=1, down_cls=0, second_cls=None): 
    if second_cls is None:
        second_cls = 1 - down_cls    
    
    # sort ys and kms
    kms_all, ys_all = sort_kms(kms, ys)
    # get boundaries, assume data is sorted
    diff = np.roll(ys_all, 1, axis=0) - ys_all
    boundaries = np.nonzero(diff)[0]
    
    # now create a downsampled (anomaly class downsample) version 
    kms, ys = downsampling_kms(kms_all,ys_all, downsampling)
    kms = [kms[i] for i in range(len(kms)) if i%gap==0]
    
    # create label for both class
    label0 = 'Inlier' if downsampling <1 else 'Class %d'% (second_cls)
    label1 = 'Outlier' if downsampling <1 else 'Class %d'%down_cls   
    color0 = 'blue' if downsampling <1 else 'tab:green'
    color1 = 'red' if downsampling <1 else 'tab:orange'
    colors = np.array([color0, color1])
    
    offset = 3 if downsampling < 1 else 0
 
    fig, axes = plt.subplots(4-offset, len(kms), figsize=(len(kms)*3,(4-offset)*3), constrained_layout=True)
    # create figure
    if downsampling < 1:
        fig.suptitle(r'\underline{\textbf{Class\ ' + str(down_cls) + '\ as\ outlier}}  with its downsampling rate ='+str(downsampling),
                    fontsize=15, usetex=True)
    else:
        fig.suptitle('Full data pairwise similarity visualization', fontsize=15)
    
    tsne = TSNE(n_components=2, metric="precomputed")
    mds = MDS(n_components=2, dissimilarity="precomputed")

    # start plot each kernel matrix
    rocs = []
    for ii, kernel_matrix in enumerate(kms):
        roc = roc_func(kernel_matrix, ys)['roc_auc'] 
        rocs.append(roc)
        percentages = {0:[], 1:[]}
        radiuses = {0:[], 1:[]}
        for i, y in enumerate(ys):
            similarities_to_other_nodes = kernel_matrix[i]
            sort_index = np.argsort(similarities_to_other_nodes)[::-1][:k] # descending order
            neighbors = ys[sort_index]
            percentage_of_abnormal = sum(neighbors!=y)/len(neighbors)
            percentages[y].append(percentage_of_abnormal)
            radius = 1 - similarities_to_other_nodes[sort_index[-1]]
            radiuses[y].append(radius)
            
        if downsampling == 1:
            aa = axes[0,ii].matshow(kernel_matrix, cmap=plt.get_cmap('RdBu').reversed())
            colorbar(aa)
            for boundary in boundaries:
                axes[0,ii].axhline(y=boundary, color='green', linestyle='-')
                axes[0,ii].axvline(x=boundary, color='green', linestyle='-')
            axes[0,ii].set_xlabel('All-graph similarity matrix')
            axes[0,ii].set_title('iter={}'.format(ii*gap+1))
            
            axes[0,ii].set_xticks([boundaries[-1]])#len(ys_all)-boundaries[-1]
            axes[0,ii].set_xticklabels(['{}  |  {}'.format(label0, label1)])
            axes[0,ii].set_yticks([boundaries[-1]])
            axes[0,ii].set_yticklabels(['{}  |  {}'.format(label1, label0)],rotation='vertical', va="center")
            
            # test
            ys =ys.astype(int)
            mds_embs = mds.fit_transform(1-kernel_matrix)
            cs = colors[ys]
            perm = np.random.permutation(len(ys))
            scatter = axes[1,ii].scatter(mds_embs[perm,0],mds_embs[perm,1], c=cs[perm], s=1, alpha=0.3)
            # produce a legend with the unique colors from the scatter
            axes[1,ii].scatter(0, 0, c=color0, s=1, label=label0, alpha=0.3)
            axes[1,ii].scatter(0, 0, c=color1, s=1, label=label1, alpha=0.3)
            axes[1,ii].legend()
            axes[1,ii].set_title('MDS visualization')
            
            sns.distplot(radiuses[0], hist=True, kde=False, color=color0, label=label0, ax=axes[2,ii])
            sns.distplot(radiuses[1], hist=True, kde=False, color=color1, label=label1, ax=axes[2,ii])

            axes[2,ii].set_xlabel('Radius of {}-NN'.format(k))
            axes[2,ii].set_xlim(0,1)
            axes[2,ii].set_ylabel('Number of graphs')
            axes[2,ii].legend()  
            
        ax = axes[3-offset,ii] if offset < 3 else axes[ii]
        if downsampling < 1:
            ax.set_title('iter={}, roc='.format(ii*gap+1)+ r'\underline{\textbf{'+ '{:.3f}'.format(roc)+'}}', usetex=True)
        sns.distplot(percentages[0], hist=False, color=color0, label=label0, ax=ax)
        sns.distplot(percentages[1], hist=False, color=color1, label=label1, ax=ax)  
        
        ax.set_xlabel('Disagree% in {}-NN'.format(k))
        ax.set_ylabel('Density of graphs'.format(k))
        ax.set_xlim(-0.1,1.1)
        ax.legend()
#     plt.tight_layout()   
    return fig, axes  

def visualize_roc_vs_iteration(kms, ys, roc_func, downsampling=1, down_cls=0, second_cls=None, repeat=5, seed=10):
    if second_cls is None:
        second_cls = 1 - down_cls 

    def plot_iters(kms, ys, downsampling, repeat, color, down_cls=0):
        kms_all, ys_all = sort_kms(kms, ys)
        runs = []
        iters = np.arange(1, len(kms_all)+1)
        for i in range(repeat):
            kms_sub, ys_sub = downsampling_kms(kms_all,ys_all, downsampling)  
            # get rocs
            rocs = []
            for ii, kernel_matrix in enumerate(kms_sub):
                rocs.append(roc_func(kernel_matrix, ys_sub)['roc_auc'])
            rocs = np.array(rocs)
            plt.plot(iters, rocs, c=color,alpha=0.2)
            runs.append(rocs)
        runs = np.stack(runs)
        plt.plot(iters, runs.mean(axis=0), c=color, label='class %d as outlier'%down_cls)    
    
    #np.random.seed(seed)
    fig = plt.figure()
    plot_iters(kms, ys, downsampling, repeat, color='red', down_cls=down_cls)
    plot_iters(kms, 1-ys, downsampling, repeat, color='blue', down_cls=second_cls)
    plt.legend()  
    plt.xlabel("Number of Propagation Iteration")
    plt.ylabel("Outlier detection ROC-AUC")
    plt.title("Outlier Downsampling Rate = {}".format(downsampling))
#     plt.tight_layout()
    
def visualize_roc_vs_sampling_rate(kms, ys, roc_func, down_cls=0, second_cls=None, iteration=5, repeat=5):
    if second_cls is None:
        second_cls = 1 - down_cls 

    def plot_rates(kms, ys, down_rates, repeat, color, down_cls):
        kms_all, ys_all = sort_kms(kms, ys)
        runs0 = []
        for i in range(repeat):
            rocs0 = []
            for down_rate in down_rates:
                kms_sub, ys_sub = downsampling_kms(kms_all, ys_all, down_rate)
                rocs0.append(roc_func(kms_sub[0], ys_sub)['roc_auc'])
            rocs0 = np.array(rocs0)
            runs0.append(rocs0)
            plt.plot(down_rates, rocs0, c=color, alpha=0.2)
        plt.plot(down_rates, np.stack(runs0).mean(0),  c=color, label='class %d as outlier'%down_cls)
        
    assert len(kms) > iteration
    down_rates = np.arange(0.05,0.9,0.05)
    kms = [kms[iteration]]
    
    fig = plt.figure()
    plot_rates(kms, ys, down_rates, repeat, color='red', down_cls=down_cls)
    plot_rates(kms, 1-ys, down_rates, repeat, color='blue', down_cls=second_cls)
    plt.legend()     
    plt.xlabel('Outlier downsampling rate')
    plt.ylabel('Outlier detection ROC-AUC')
    plt.title('Propagation iteration = {}'.format(iteration))
#     plt.tight_layout()
    
from torch_geometric.data import DataLoader     
def investigate_kernel(data_name, kernel_name, detector_name, n, down_cls=1, second_cls=None, labeled=True, bin_width=0.1, anomaly_downsampling_rate=0.1, seed=15213):
    down_rate = 1
    k = 20

    if second_cls is None:
        second_cls = 1 - down_cls 

    # Load data, seed is fixed here
    dataset = load_data(data_name, down_class=down_cls, down_rate=down_rate, second_class=second_cls, seed=seed)[2] 
    
    # after this, the class label has been changed: 0-second_cls  1-down_cls
    ys = torch.cat([data.y for data in dataset])

    # create loader
    loader = DataLoader(dataset, batch_size=1, shuffle=False)

    niter = n if kernel_name =='PK' else n - 1

    # build model
    model = KernelBasedGLAD(kernel=kernel_name, detector=detector_name, WL_iter=niter, 
                            PK_bin_width=bin_width, labeled=labeled, LOF_n_neighbors=20)
    result = model.fit(loader)
    kernel_matrices, accumulative_kernel_matrices = model.get_kernel_matrices()

    # build saving directory 
    result_dir = os.path.join(kernel_name, data_name+f'-{down_cls}-{second_cls}', detector_name)  # considering the downsample part     
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)  

    # visualize maps
    mode = 'accumulative'
    kms = kernel_matrices if mode == 'separate' else accumulative_kernel_matrices
    
    gap=2

    # visualize maps, full data
    visualize_disperity(kms, ys.numpy(), model.fit_kernel_matrix, k=k, gap=gap, downsampling=1, down_cls=down_cls) 
    plt.savefig(os.path.join(result_dir,'overlap_disperity-fulldata-{}-{}-{}-bin{}.'.format(
                                data_name, kernel_name, detector_name, bin_width)+ format), format=format)
    plt.close()

    # visualize maps downsampling = 0.1 with down_cls=0
    visualize_disperity(kms, ys.numpy(), model.fit_kernel_matrix, k=k, gap=gap,
                        downsampling=anomaly_downsampling_rate, down_cls=down_cls)
    plt.savefig(os.path.join(result_dir,'overlap_disperity-downsampled(c{}-r{})-{}-{}-{}-bin{}.'.format(
        down_cls, anomaly_downsampling_rate, data_name, kernel_name, detector_name, bin_width)+ format), format = format)
    plt.close()
    
    # visualize maps downsampling = 0.1 with down_cls=1
    # filp data label
    visualize_disperity(kms, 1-ys.numpy(), model.fit_kernel_matrix, k=k, gap=gap,
                        downsampling=anomaly_downsampling_rate, down_cls=second_cls)
    plt.savefig(os.path.join(result_dir,'overlap_disperity-downsampled(c{}-r{})-{}-{}-{}-bin{}.'.format(
        second_cls, anomaly_downsampling_rate, data_name, kernel_name, detector_name, bin_width)+format), format = format) 
    plt.close()
    
    # visualize roc-vs-iteration
    visualize_roc_vs_iteration(kms, ys, model.fit_kernel_matrix, anomaly_downsampling_rate, down_cls, repeat=10)
    plt.savefig(os.path.join(result_dir,'roc_vs_iter-{}-{}-{}.'.format(data_name, kernel_name, detector_name)+format), format = format) 
    plt.close()
    
    # visualize roc-vs-sampling_rate
    visualize_roc_vs_sampling_rate(kms, ys, model.fit_kernel_matrix, down_cls, repeat=10)   
    plt.savefig(os.path.join(result_dir,'roc_vs_downrate-{}-{}-{}.'.format(data_name, kernel_name, detector_name)+format), format = format) 
    plt.close()

# investigate_kernel('PROTEINS', 'PK', 'LOF', 11)

### -Experiments: Visualization

In [ ]:

# datasets = ['DD', 'PROTEINS', 'NCI1', 'IMDB-BINARY', 'Mutagenicity', 'PTC_MR', 'ENZYMES', 'COLLAB']
# data_name='PROTEINS'
# kernel_name = 'WL'
detector_name = 'LOF'
n=11      
# 'DD', 'PROTEINS', 'NCI1', 
for data in ['IMDB-BINARY', 'Mutagenicity', 'AIDS', 'ENZYMES-0-1', 'ENZYMES-2-3', 'COLLAB-0-1', 'COLLAB-1-2']:
    d = data.split('-')
    if len(d) == 2:
        # 'IMDB-BINARY'
        d[0] = d[0] + '-' + d[1] 
        down_cls, second_cls = 1, 0
    elif len(d) > 2:
        down_cls, second_cls = int(d[1]), int(d[2])
    else:
        down_cls, second_cls = 1, 0
    for kernel_name in ['WL' , 'PK']: #'PK'
        investigate_kernel(d[0], kernel_name, detector_name, n, down_cls=down_cls, second_cls=second_cls, seed=15213)

### -Experiments: 10 seeds, 5-layer, 0.1 down-rate 

In [ ]:
data_name, down_cls, second_cls, seed = 'REDDIT-MULTI-5K', 2, 0, 15213
kernel_name = 'WL'

# need to test bin_width=0.1

dataset = load_data(data_name, down_class=down_cls, second_class=second_cls, down_rate=0.1, seed=seed)[2] 
loader = DataLoader(dataset, batch_size=1, shuffle=False)
model = KernelBasedGLAD(kernel=kernel_name, detector='LOF', WL_iter=5, PK_bin_width=0.01)
results = model.fit(loader) 
print(results)

In [ ]:
import logging
def run(data_name, kernel_name, detector_name='LOF', seed=15213, down_rate=0.1, down_cls=0, second_cls=None):
    # Reset logging: Remove all handlers associated with the root logger object.
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    logging.basicConfig(format='%(message)s', level=logging.INFO, filename=f'results/{data_name}-{kernel_name}-{detector_name}.log')
    description = f'!Data[{data_name}] DownClass[{down_cls}] SecondClass[{second_cls}] Model[{kernel_name}-{detector_name}] Seed[{seed}]' 
    logging.info('-'*50)
    logging.info(description)
    dataset = load_data(data_name, down_class=down_cls, second_class=second_cls, down_rate=down_rate, seed=seed)[2] 
    loader = DataLoader(dataset, batch_size=1, shuffle=False)
    model = KernelBasedGLAD(kernel=kernel_name, detector=detector_name, WL_iter=5, PK_bin_width=0.1)
    results = model.fit(loader) 
    logging.info(results)



detector_name = 'OCSVM'
seeds = [15213, 10086, 11777, 333, 444, 555, 666, 777, 888, 999]
datasets = ['DD', 'PROTEINS', 'NCI1', 'IMDB-BINARY', 'Mutagenicity', 'AIDS', 'ENZYMES-0-1', 'ENZYMES-2-3', 'COLLAB-0-1', 'COLLAB-1-2'] 

datasets = ['REDDIT-MULTI-5K-0-1', 'REDDIT-MULTI-5K-3-4', 'REDDIT-MULTI-5K-2-3']
# datasets = ['REDDIT-MULTI-5K-2-3']

for data in datasets:
    d = data.split('-')
    if len(d) == 2:
        # 'IMDB-BINARY'
        d[0] = d[0] + '-' + d[1] 
        down_cls, second_cls = 0, 1
    elif len(d) > 2:
        d[0] = '-'.join(d[:-2])
        down_cls, second_cls = int(d[-2]), int(d[-1])
    else:
        down_cls, second_cls = 0, 1
    for kernel in  ['WL']:#['PK','WL']: # ['WL', 'PK']:
        for detector_name in ['LOF', 'OCSVM']:
            for seed in seeds:
                run(d[0], kernel, detector_name, seed=seed, down_cls=down_cls, second_cls=second_cls)
            for seed in seeds:
                run(d[0], kernel, detector_name, seed=seed, down_cls=second_cls, second_cls=down_cls)

# Graph2Vec FSGD Experiments
### -Model 

In [ ]:
from embedder import *

### -Experiments: benchmark, 10 seeds, 0.1 down-rate

In [ ]:
import logging
def run(data_name, embedder_name, detector_name='LOF', seed=15213, down_rate=0.1, down_cls=0, second_cls=None):
    # Reset logging: Remove all handlers associated with the root logger object.
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    logging.basicConfig(format='%(message)s', level=logging.INFO, filename=f'results/{data_name}-{embedder_name}-{detector_name}.log')
    description = f'!Data[{data_name}] DownClass[{down_cls}] SecondClass[{second_cls}] Model[{embedder_name}-{detector_name}] Seed[{seed}]' 
    logging.info('-'*50)
    logging.info(description)
    dataset = load_data(data_name, down_class=down_cls, second_class=second_cls, down_rate=down_rate, seed=seed)[2] 
    loader = DataLoader(dataset, batch_size=1, shuffle=False)
    model = EmbeddingBasedGLAD(embedder=embedder_name, detector=detector_name, G2V_wl_iter=3, normalize_embedding=False)
    results = model.fit(loader)
    print(results)
    logging.info(results)


detector_name = 'OCSVM'
seeds = [15213, 10086, 11777, 333, 444, 555, 666, 777, 888, 999]
datasets = ['DD', 'PROTEINS', 'NCI1', 'IMDB-BINARY', 'Mutagenicity', 'AIDS', 'ENZYMES-0-1', 'ENZYMES-2-3', 'COLLAB-0-1', 'COLLAB-1-2']

datasets = ['REDDIT-MULTI-5K-0-1', 'REDDIT-MULTI-5K-3-4','REDDIT-MULTI-5K-2-3']
#datasets = ['COLLAB-1-2', 'REDDIT-MULTI-5K-2-3']


for data in datasets:
    d = data.split('-')
    if len(d) == 2:
        # 'IMDB-BINARY'
        d[0] = d[0] + '-' + d[1] 
        down_cls, second_cls = 0, 1
    elif len(d) > 2:
        d[0] = '-'.join(d[:-2])
        down_cls, second_cls = int(d[-2]), int(d[-1])
    else:
        down_cls, second_cls = 0, 1
    for kernel in  ['Graph2Vec']: # ['Graph2Vec', 'FGSD']: # ['WL', 'PK']:
        for detector_name in ['IF']: # ['LOF', 'OCSVM']:
            for seed in seeds:
                run(d[0], kernel, detector_name, seed=seed, down_cls=down_cls, second_cls=second_cls)
            for seed in seeds:
                run(d[0], kernel, detector_name, seed=seed, down_cls=second_cls, second_cls=down_cls)

### -Visualization

In [ ]:
# Step 1: T-SNE embedding visualization
from sklearn.manifold import TSNE, MDS
def investigate_embedder(data_name, embedder_name, detector_name='LOF', down_class=1, second_class=0, seed=15213):
    k=20
    if second_class is None:
        second_class = 1 - down_class 

    # Load data, seed is fixed here
    dataset = load_data(data_name, down_class=down_class, second_class=second_class, down_rate=1, seed=seed)[2] 
    
    # after this, the class label has been changed: 0-second_cls  1-down_cls
    ys = torch.cat([data.y for data in dataset]).numpy().astype(int)

    # create loader
    loader = DataLoader(dataset, batch_size=1, shuffle=False)

    # build model
    model = EmbeddingBasedGLAD(embedder=embedder_name, detector=detector_name, G2V_wl_iter=3)
    result = model.fit(loader)
    embeds =  model.embedder.get_embedding()

    # generate pairwise similarity matrix
    # 1. sort embeds centers and ys
    order = np.argsort(ys)
    ys = ys[order]
    embeds = embeds[order,:]

    # 2. get boundaries, assume data is sorted
    diff = np.roll(ys, 1, axis=0) - ys
    boundaries = np.nonzero(diff)[0]
    # 3. calculate pairwise similarity
    kernel_matrix = euclidean(embeds)

    # create label for both class
    label0 = 'Class %d'% second_class
    label1 = 'Class %d'% down_class
    color0 = 'tab:green'
    color1 = 'tab:orange'
    colors = np.array([color0, color1])

    percentages = {0:[], 1:[]}
    radiuses = {0:[], 1:[]}
    # overlap counting
    for i, y in enumerate(ys):
        similarities_to_other_nodes = kernel_matrix[i]
        sort_index = np.argsort(similarities_to_other_nodes)[::-1][:k] # descending order
        neighbors = ys[sort_index]
        percentage_of_abnormal = sum(neighbors!=y)/len(neighbors)
        percentages[y].append(percentage_of_abnormal)
        radius = 1 - similarities_to_other_nodes[sort_index[-1]]
        radiuses[y].append(radius)

    plt.figure(figsize=(5*4, 5))
    # plot matrix
    plt.subplot(1,4,1)
    ax = plt.gca()
    colorbar(ax.matshow(kernel_matrix, cmap=plt.get_cmap('RdBu').reversed()))
    for boundary in boundaries:
        plt.axhline(y=boundary, color='green', linestyle='-')
        plt.axvline(x=boundary, color='green', linestyle='-')
    plt.xlabel('All-graph similarity matrix')
    ax.set_xticks([boundaries[-1]])#len(ys_all)-boundaries[-1]
    ax.set_xticklabels(['{}  |  {}'.format(label0, label1)])
    ax.set_yticks([boundaries[-1]])
    ax.set_yticklabels(['{}  |  {}'.format(label1, label0)],rotation='vertical', va="center")

    # MDS visualization
    plt.subplot(1,4,2)
    mds = MDS(n_components=2, dissimilarity="precomputed")
    mds_embs = mds.fit_transform(1-kernel_matrix)
    cs = colors[ys]
    perm = np.random.permutation(len(ys))
    plt.scatter(mds_embs[perm,0],mds_embs[perm,1], c=cs[perm], s=1, alpha=0.3)
    # produce a legend with the unique colors from the scatter
    plt.scatter(0, 0, c=color0, s=1, label=label0, alpha=0.3)
    plt.scatter(0, 0, c=color1, s=1, label=label1, alpha=0.3)
    plt.legend()
    plt.title('MDS visualization')

    plt.subplot(1,4,3)
    ax = plt.gca()
    sns.distplot(radiuses[0], hist=True, kde=False, color=color0, label=label0, ax=ax)
    sns.distplot(radiuses[1], hist=True, kde=False, color=color1, label=label1, ax=ax)
    ax.set_xlabel('Radius of {}-NN'.format(k))
    ax.set_xlim(0,1)
    ax.set_ylabel('Number of graphs')
    ax.legend()  
        
    plt.subplot(1,4,4)
    ax = plt.gca()
    sns.distplot(percentages[0], hist=False, color=color0, label=label0, ax=ax)
    sns.distplot(percentages[1], hist=False, color=color1, label=label1, ax=ax)  
    ax.set_xlabel('Disagree% in {}-NN'.format(k))
    ax.set_ylabel('Density of graphs'.format(k))
    ax.set_xlim(-0.1,1.1)
    ax.legend()
    plt.tight_layout()


In [ ]:
# seeds = [15213, 10086, 11777, 333, 444, 555, 666, 777, 888, 999]
datasets = ['DD', 'PROTEINS', 'NCI1', 'IMDB-BINARY', 'Mutagenicity', 'AIDS', 'ENZYMES-0-1', 'ENZYMES-2-3', 'COLLAB-0-1', 'COLLAB-1-2']
detector_name = 'LOF'
for data in datasets:
    d = data.split('-')
    if len(d) == 2:
        # 'IMDB-BINARY'
        d[0] = d[0] + '-' + d[1] 
        down_cls, second_cls = 1, 0
    elif len(d) > 2:
        down_cls, second_cls = int(d[1]), int(d[2])
    else:
        down_cls, second_cls = 1, 0
    for kernel in  ['Graph2Vec', 'FGSD']: 
        result_dir = os.path.join(kernel, data, detector_name)  # considering the downsample part  
        if not os.path.exists(result_dir):
            os.makedirs(result_dir)  
        investigate_embedder(d[0], kernel, detector_name, down_cls, second_cls)
        plt.savefig(os.path.join(result_dir,'overlap_disperity-fulldata-{}-{}-{}.'.format(
                                    data, kernel, detector_name)+ format), format=format)
        plt.close()

In [ ]:
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import RobustScaler, StandardScaler, Normalizer
# # PCA(n_components=10).fit_transform(embeds)
# kernel_matrix = euclidean(embeds/np.linalg.norm(embeds, axis=1, keepdims=True))
# # kernel_matrix = euclidean(RobustScaler().fit_transform(embeds))
# # kernel_matrix = euclidean(StandardScaler().fit_transform(embeds))
# # kernel_matrix = euclidean(Normalizer().fit_transform(embeds))

# OCGIN Experiments
### -Model

In [ ]:
from ocgin import *

### -Train Model and Save

In [ ]:
# Train model and save it down
import logging
import os, numpy as np
import shutil
import pytorch_lightning as pl
    
def train_ocgin(data_name, down_class, nlayer, second_class=None, max_epoch=25, seed=1213, down_rate=0.1):
    # create loader
    loaders = create_loaders(data_name, batch_size=32, down_class=down_class, second_class=second_class, down_rate=down_rate, seed=seed)

    # create model
    model = OCGIN(loaders[3][0].num_features, weight_decay=5e-4, nlayer=nlayer)
    # setup
    if second_class is None:
        second_class = 1 - down_class
    
    working_dir = os.path.join(type(model).__name__,
                               f'{data_name}-{down_class}-{second_class}',
                               'nlayer-%d'%nlayer,
                               'seed-%d'%seed)    
    shutil.rmtree(working_dir, ignore_errors=True)

    # Reset logging: Remove all handlers associated with the root logger object.
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    # logging.getLogger('lightning').setLevel(0)
    logging.basicConfig(format='%(message)s', level=logging.INFO, filename=f'results/{data_name}-OCGIN.log')
    description = f'!Data[{data_name}] DownClass[{down_class}] SecondClass[{second_class}] Model[OCGIN-{nlayer}] Seed[{seed}]' 
    logging.info('-'*50)
    logging.info(description)

    # save initialized model, and save center
    trainer0 = pl.Trainer(gpus=1, max_epochs=1, logger=False, weights_summary=None) # 1 epcoh to initialize the center of GIN
    trainer0.fit(model, loaders[0]) 
    result0 = trainer0.test(test_dataloaders=loaders[2])[0]
    trainer0.save_checkpoint(working_dir+'/epoch=0-val_roc_auc={:.3f}.ckpt'.format(result0['roc_auc']))

    # train
    trainer = pl.Trainer(gpus=1, max_epochs=max_epoch, default_root_dir=working_dir, weights_summary=None)
    
    trainer.fit(model, loaders[0]) # only use training set, no validation is used
    result25 = trainer.test(test_dataloaders=loaders[2])[0]
    trainer.save_checkpoint(working_dir+'/epoch=25-val_roc_auc={:.3f}.ckpt'.format(result25['roc_auc']))
    logging.info("epoch0:"+str(result0))
    logging.info(result25)

In [ ]:
seeds = [15213, 10086, 11777, 333, 444, 555, 666, 777, 888, 999]
datasets = ['DD', 'PROTEINS', 'NCI1', 'IMDB-BINARY', 'Mutagenicity', 'AIDS', 'ENZYMES-0-1', 'ENZYMES-2-3', 'COLLAB-0-1', 'COLLAB-1-2']

datasets = ['REDDIT-BINARY', 'REDDIT-MULTI-5K-0-1', 'REDDIT-MULTI-5K-3-4', 'REDDIT-MULTI-5K-1-3']

for data in datasets:
    d = data.split('-')
    if len(d) == 2:
        # 'IMDB-BINARY'
        d[0] = d[0] + '-' + d[1] 
        down_cls, second_cls = 0, 1
    elif len(d) > 2:
        d[0] = '-'.join(d[:-2])
        down_cls, second_cls = int(d[-2]), int(d[-1])
    else:
        down_cls, second_cls = 0, 1

    for seed in seeds:
        train_ocgin(d[0], down_class=down_cls, second_class=second_cls, nlayer=5, seed=seed)
    for seed in seeds:
        train_ocgin(d[0], down_class=second_cls, second_class=down_cls, nlayer=5, seed=seed)

### -Visualization Tools

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.manifold import TSNE, MDS

def euclidean(embeddings):
    #embeddings = embeddings - embeddings.mean(0, keepdims=True)
    distances = euclidean_distances(embeddings, embeddings, squared=False)
    distances /= distances.max() # between 0 and 1
    return 1 - distances # similarity

def sort_embeds(embeds, ys):
    order = np.argsort(ys)
    ys = ys[order]
    embeds = embeds[:, order,:]
    return embeds, ys

def downsampling_embeds(embeds, ys, down_rate=1):
    # this need to called after sort
    index_anomaly = np.where(ys==1)[0]
    sub_index_anomaly = np.random.choice(index_anomaly, int(len(index_anomaly)*down_rate), replace=False) 
    sub_index = np.concatenate([np.where(ys==0)[0], sub_index_anomaly])
    ys_sub = ys[sub_index]
    embeds_sub =  embeds[:, sub_index, :] 
    return embeds_sub, ys_sub

def get_performance(embed, ys, center):
    anomaly_scores = ((embed - center)**2).sum(1)
    return roc_auc_score(ys, anomaly_scores)

def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    import matplotlib.pyplot as plt
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

In [ ]:
import glob
def load_model_get_embedding(data_name, down_class, second_class, nlayer, epoch, seed, Model=OCGIN, down_rate=1):
    working_dir = os.path.join(Model.__name__, f'{data_name}-{down_class}-{second_class}',
                               'nlayer-%d'%nlayer, 'seed-%d'%seed)                                
    ckpt = glob.glob(os.path.join(working_dir,'*epoch=%d-*'%epoch))[0]
    m = Model.load_from_checkpoint(ckpt)
    m.eval()
    m.cuda()
    
    # load all data without downsampling, but do set anomaly class as down_class
    dataset = load_data(data_name, down_class=down_class, second_class=second_class, down_rate=down_rate, seed=seed)[2]
    ys = torch.cat([data.y for data in dataset])
    loader = DataLoader(dataset, batch_size=32, shuffle=False)
    
    # pass data into model to get embeddings
    with torch.no_grad():
        embeds, ys = [], []
        for data in loader:
            #batch_embeds, batch_stds = m(data.to(device))
            embeds.append(m.get_hiddens(data.to(torch.device("cuda"))))
            ys.append(data.y)
            
    embeds = torch.cat(embeds, dim=1).cpu().numpy()
    embeds = np.add.accumulate(embeds, axis=0)
    ys = torch.cat(ys).cpu().numpy()
    
    # get centers for each accumulative embeddings
    centers = m.all_layer_centers.cpu().numpy()
    centers = np.add.accumulate(centers, axis=0)
    return embeds, ys, centers


def visualize_disperity_full(data_name, down_class, second_class, nlayer, epoch, seed, Model=OCGIN, gap=1):
    embeds, ys, centers = load_model_get_embedding(data_name, down_class, second_class, nlayer, epoch, seed, Model=Model, down_rate=1)
    roc_func=get_performance
    k=20

    # sort embeds centers and ys
    embeds_all, ys_all = sort_embeds(embeds, ys)
     
    # get boundaries, assume data is sorted
    diff = np.roll(ys_all, 1, axis=0) - ys_all
    boundaries = np.nonzero(diff)[0]
    
    # now create a downsampled (anomaly class downsample) version 
    embeds_sub, ys_sub = downsampling_embeds(embeds_all, ys_all, 1)
    
    # only evaluate the one between gap
    embeds_sub = np.stack([embeds_sub[i] for i in range(len(embeds_sub)) if i%gap==0])
    centers_sub = np.stack([centers[i] for i in range(len(centers)) if i%gap==0])
    
    # create label for both class
    label0 = 'Class %d'% second_class
    label1 = 'Class %d'% down_class
    color0 = 'tab:green'
    color1 = 'tab:orange'
    colors = np.array([color0, color1])
    
    offset = 0
    fig, axes = plt.subplots(4-offset, len(embeds_sub), figsize=(len(embeds_sub)*3,(4-offset)*3), constrained_layout=True)
    # create figure
    fig.suptitle('Full data pairwise similarity visualization',fontsize=15)
          
    # visualize
    for ii, embed in enumerate(embeds_sub):
        roc = roc_func(embed, ys_sub, centers_sub[ii])
        percentages = {0:[], 1:[]}
        radiuses = {0:[], 1:[]}
        # calculate similarity matrix
        kernel_matrix = euclidean(embed)
        kernel_matrix_all = euclidean(embeds_all[ii*gap])
        
        # overlap counting
        for i, y in enumerate(ys_sub):
            similarities_to_other_nodes = kernel_matrix[i]
            sort_index = np.argsort(similarities_to_other_nodes)[::-1][:k] # descending order
            neighbors = ys_sub[sort_index]
            percentage_of_abnormal = sum(neighbors!=y)/len(neighbors)
            percentages[y].append(percentage_of_abnormal)
            radius = 1 - similarities_to_other_nodes[sort_index[-1]]
            radiuses[y].append(radius)
            
    
        # plot matrix
        aa = axes[0,ii].matshow(kernel_matrix_all, cmap=plt.get_cmap('RdBu').reversed())
        colorbar(aa)
        for boundary in boundaries:
            axes[0,ii].axhline(y=boundary, color='green', linestyle='-')
            axes[0,ii].axvline(x=boundary, color='green', linestyle='-')
        axes[0,ii].set_title('#layer={}'.format(ii*gap))
        axes[0,ii].set_xlabel('All-graph similarity matrix')
        
        axes[0,ii].set_xticks([boundaries[-1]])#len(ys_all)-boundaries[-1]
        axes[0,ii].set_xticklabels(['{}  |  {}'.format(label0, label1)])
        axes[0,ii].set_yticks([boundaries[-1]])
        axes[0,ii].set_yticklabels(['{}  |  {}'.format(label1, label0)],rotation='vertical', va="center")
        
        # mds embedding           
        ys = ys.astype(int)
        mds = MDS(n_components=2, dissimilarity="precomputed")
        mds_embs = mds.fit_transform(1-kernel_matrix)

        # mds = MDS(n_components=2)
        # tsne = TSNE(n_components=2)
        # mds_embs = tsne.fit_transform(embed.astype(np.float64))

        cs = colors[ys]
        perm = np.random.permutation(len(ys))
        axes[1,ii].scatter(mds_embs[perm,0],mds_embs[perm,1], c=cs[perm], s=1, alpha=0.3)
        # produce a legend with the unique colors from the scatter
        axes[1,ii].scatter(0, 0, c=color0, s=1, label=label0, alpha=0.3)
        axes[1,ii].scatter(0, 0, c=color1, s=1, label=label1, alpha=0.3)
        axes[1,ii].legend()
        axes[1,ii].set_title('MDS visualization')
        
        sns.distplot(radiuses[0], hist=True,kde=False, color=color0, label=label0, ax=axes[2,ii])
        sns.distplot(radiuses[1], hist=True,kde=False, color=color1, label=label1, ax=axes[2,ii])
        axes[2,ii].set_xlabel('Radius of {}-NN'.format(k))
        axes[2,ii].set_xlim(0,1)
        axes[2,ii].set_ylabel('Number of graphs')
        axes[2,ii].legend()  
            
        ax = axes[3-offset,ii] if offset < 3 else axes[ii]
        sns.distplot(percentages[0], hist=False, color=color0, label=label0, ax=ax)
        sns.distplot(percentages[1], hist=False, color=color1, label=label1, ax=ax)  
        
        ax.set_xlabel('Disagree% in {}-NN'.format(k))
        ax.set_ylabel('Density of graphs)'.format(k))
        ax.set_xlim(-0.1,1.1)
        ax.legend()
    # plt.tight_layout()

def visualize_disperity_downsampled(data_name, down_class, second_class, nlayer, epoch, seed, Model=OCGIN, gap=1):
    embeds, ys, centers = load_model_get_embedding(data_name, down_class, second_class, nlayer, epoch, seed, Model=Model, down_rate=0.1)
    # sort embeds centers and ys
    embeds, ys = sort_embeds(embeds, ys)
    k=20
     
    # get boundaries, assume data is sorted
    diff = np.roll(ys, 1, axis=0) - ys
    boundaries = np.nonzero(diff)[0]
    
    # create label for both class
    label0, label1, color0, color1 = 'Inlier', 'Outlier', 'blue', 'red' 
    colors = np.array([color0, color1])
    
    embeds = np.stack([embeds[i] for i in range(len(embeds)) if i%gap==0])
    centers = np.stack([centers[i] for i in range(len(centers)) if i%gap==0])
    
    fig, axes = plt.subplots(1, len(embeds), figsize=(len(embeds)*3,3), constrained_layout=True)
    fig.suptitle(r'\underline{\textbf{Class\ ' + str(down_class) + '\ as\ outlier}}  with its downsampling rate=0.1',
                    fontsize=15, usetex=True)   
      
    # get performance
    rocs = []
    for ii, embed in enumerate(embeds):
        roc = get_performance(embed, ys, centers[ii])
        rocs.append(roc)
        percentages = {0:[], 1:[]}
        radiuses = {0:[], 1:[]}
        # calculate similarity matrix
        kernel_matrix = euclidean(embed)        
         # overlap counting
        for i, y in enumerate(ys):
            similarities_to_other_nodes = kernel_matrix[i]
            sort_index = np.argsort(similarities_to_other_nodes)[::-1][:k] # descending order
            neighbors = ys[sort_index]
            percentage_of_abnormal = sum(neighbors!=y)/len(neighbors)
            percentages[y].append(percentage_of_abnormal)
            radius = 1 - similarities_to_other_nodes[sort_index[-1]]
            radiuses[y].append(radius)
        ax =  axes[ii]    
        ax.set_title('\#layer={}, roc='.format(ii*gap)+ r'\underline{\textbf{'+ '{:.3f}'.format(roc)+'}}', usetex=True)
        sns.distplot(percentages[0], hist=False, color=color0, label=label0, ax=ax)
        sns.distplot(percentages[1], hist=False, color=color1, label=label1, ax=ax)  

        ax.set_xlabel('Disagree% in {}-NN'.format(k))
        ax.set_ylabel('Density of graphs'.format(k))
        ax.set_xlim(-0.1,1.1)
        ax.legend()


def load_model_get_performance_perseed(data_name, down_class, second_class, nlayer, epoch, seed, Model=OCGIN, down_rate=0.1):
    embeds, ys, centers = load_model_get_embedding(data_name, down_class, second_class, nlayer, epoch, seed, Model=Model, down_rate=down_rate)
    # get performance
    rocs = []
    for ii, embed in enumerate(embeds):
        roc = get_performance(embed, ys, centers[ii])
        rocs.append(roc)
    return rocs

def load_model_get_performance(data_name, down_class, second_class, epoch, nlayer=5, mean=True, down_rate=0.1):
    seeds = [15213, 10086, 11777, 333, 444, 555, 666, 777, 888, 999]
    rocs = []
    for seed in seeds:
        rocs_seed = load_model_get_performance_perseed(data_name, down_class, second_class, nlayer, epoch, seed, down_rate=down_rate)   
        rocs.append(rocs_seed)
    rocs = np.array(rocs)
    if mean is True:
        rocs = rocs.mean(axis=0)
    return rocs

def visualize_roc_vs_iteration(data_name, down_class, second_class, nlayer=5, epoch=25, down_rate=0.1):
    fig = plt.figure()
    rocs0 = load_model_get_performance(data_name, down_class, second_class, epoch, mean=False, down_rate=down_rate)
    rocs1 = load_model_get_performance(data_name, second_class, down_class, epoch, mean=False, down_rate=down_rate)
    for roc in rocs0:
        plt.plot(roc, c='red', alpha=0.2)
    plt.plot(rocs0.mean(axis=0), c='red', label='class %d as outlier'%down_class) 
    for roc in rocs1:
        plt.plot(roc, c='blue', alpha=0.2)
    plt.plot(rocs1.mean(axis=0), c='blue', label='class %d as outlier'%second_class) 

    plt.legend()  
    plt.xlabel("Number of layers")
    plt.ylabel("Outlier detection ROC-AUC")
    plt.title("Outlier Downsampling Rate = {}".format(down_rate))

def visualize_roc_vs_sampling_rate(data_name, down_class, second_class):
    # This function needs running train_ocgin_without_save first
    # read results from file
    down_rates = np.linspace(0.05,0.85,17)
    file_name = os.path.join('results', f'{data_name}-OCGIN-different_down_rate.log')
    results = {rate:{} for rate in down_rates} 
    with open(file_name, 'r') as file:
        lines = filter(lambda x:x[0]=='[' or x[0]=='!' or x[0]=='{', file.readlines())
        for line in lines:
            line = line.split()
            if len(line) == 6:
                line = [x.split('[') for x in line]
                line = {x[0]: x[1].strip(']') for x in line}
                key = line['!Data'] + '-' +  line['DownClass'] + '-' + line['SecondClass']
                rate = float(line['DownRate'])
                if not key in results[rate]:
                    results[rate][key] = []
            else:
                results[rate][key].append(float(line[1].strip(',') )) 
    rocs = np.zeros((2, len(down_rates), 10))
    for i, rate in enumerate(down_rates):
        rocs[0, i] = np.array(results[rate][f'{data_name}-{down_class}-{second_class}'])
        rocs[1, i] = np.array(results[rate][f'{data_name}-{second_class}-{down_class}'])
    # plot 
    plt.figure()
    rocs0 = rocs[0].T
    rocs1 = rocs[1].T
    for roc in rocs0:
        plt.plot(down_rates, roc, c='red', alpha=0.2)
    plt.plot(down_rates, rocs0.mean(axis=0), c='red', label='class %d as outlier'%down_class) 
    for roc in rocs1:
        plt.plot(down_rates, roc, c='blue', alpha=0.2)
    plt.plot(down_rates, rocs1.mean(axis=0), c='blue', label='class %d as outlier'%second_class) 
    plt.legend()
    plt.xlabel('Outlier downsampling rate')
    plt.ylabel('Outlier detection ROC-AUC')
    plt.title('Number of layer = {}'.format(5)) 

def investigate_model_embeddings(data_name, down_class, second_class, nlayer=5, epoch=25, seed=15213, Model=OCGIN):
    # define result saving path
    result_dir = os.path.join('OCGIN_Plots', f'{data_name}-{down_class}-{second_class}', f'nlayer{nlayer}')
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
    
    gap=1
    # visualize full data
    
    visualize_disperity_full(data_name, down_class, second_class, nlayer, epoch, seed, gap=gap)
    plt.savefig(os.path.join(result_dir, 'overlap_disperity-fulldata-{}-{}-epoch{}-downcls{}.'.format(
                    data_name, Model.__name__, epoch, 1)+ format), format=format)
    plt.close()
    # visualize downsampled data (transductive)
    visualize_disperity_downsampled(data_name, down_class, second_class, nlayer, epoch, seed, gap=gap)
    plt.savefig(os.path.join(result_dir,'overlap_disperity-downsampled(c{}-r{})-{}-{}-epoch{}.'.format(
                        down_class, 0.1, data_name, Model.__name__, epoch)+ format),format=format)
    plt.close()
    
    visualize_disperity_downsampled(data_name, second_class, down_class, nlayer, epoch, seed, gap=gap)
    plt.savefig(os.path.join(result_dir,'overlap_disperity-downsampled(c{}-r{})-{}-{}-epoch{}.'.format(
                        second_class, 0.1, data_name, Model.__name__, epoch)+ format), format=format)
    plt.close()

    # visualize roc-vs-iteration  down_rate=0.1
    visualize_roc_vs_iteration(data_name, down_class, second_class, nlayer, epoch)
    plt.savefig(os.path.join(result_dir,'roc_vs_iter-{}-{}-epoch{}.'.format(data_name, Model.__name__,epoch)+format), format=format)
    plt.close()

    # visualize roc-vs-samplingrate
    visualize_roc_vs_sampling_rate(data_name, down_class, second_class)
    plt.savefig(os.path.join(result_dir,'roc_vs_downrate-{}-{}-epoch{}.'.format(data_name, Model.__name__,epoch)+format), format=format)
    plt.close()

### -Experiments: visualization, roc-vs-sampling_rate

In [ ]:
#  roc-vs-samplingrate
import logging
import os, numpy as np
import shutil
import pytorch_lightning as pl
    
def train_ocgin_without_save(data_name, down_class, nlayer, second_class=None, max_epoch=25, seed=1213, down_rate=0.1):
    # create loader
    loaders = create_loaders(data_name, batch_size=32, down_class=down_class, second_class=second_class, down_rate=down_rate, seed=seed)

    # create model
    model = OCGIN(loaders[3][0].num_features, weight_decay=5e-4, nlayer=nlayer)
    # setup
    if second_class is None:
        second_class = 1 - down_class

    # Reset logging: Remove all handlers associated with the root logger object.
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    # logging.getLogger('lightning').setLevel(0)
    logging.basicConfig(format='%(message)s', level=logging.INFO, filename=f'results/{data_name}-OCGIN-different_down_rate.log')
    description = f'!Data[{data_name}] DownClass[{down_class}] SecondClass[{second_class}] Model[OCGIN-{nlayer}] DownRate[{down_rate}] Seed[{seed}]' 
    logging.info('-'*50)
    logging.info(description)

    # save initialized model, and save center
    trainer0 = pl.Trainer(gpus=1, max_epochs=1, logger=False, weights_summary=None) # 1 epcoh to initialize the center of GIN
    trainer0.fit(model, loaders[0]) 
    result0 = trainer0.test(test_dataloaders=loaders[2])[0]

    # train
    trainer = pl.Trainer(gpus=1, max_epochs=max_epoch, logger=False,  weights_summary=None)
    trainer.fit(model, loaders[0]) # only use training set, no validation is used
    result25 = trainer.test(test_dataloaders=loaders[2])[0]
    logging.info("epoch0:"+str(result0))
    logging.info(result25)

seeds = [15213, 10086, 11777, 333, 444, 555, 666, 777, 888, 999]
datasets = ['DD', 'PROTEINS', 'NCI1', 'IMDB-BINARY', 'Mutagenicity', 'AIDS', 'ENZYMES-0-1', 'ENZYMES-2-3', 'COLLAB-0-1', 'COLLAB-1-2']
for data in datasets:
    d = data.split('-')
    if len(d) == 2:
        # 'IMDB-BINARY'
        d[0] = d[0] + '-' + d[1] 
        down_cls, second_cls = 0, 1
    elif len(d) > 2:
        down_cls, second_cls = int(d[1]), int(d[2])
    else:
        down_cls, second_cls = 0, 1
    for down_rate in np.linspace(0.05,0.85,17):
        for seed in seeds:
            train_ocgin_without_save(d[0], down_class=down_cls, second_class=second_cls, nlayer=5, seed=seed, down_rate=down_rate)
        for seed in seeds:
            train_ocgin_without_save(d[0], down_class=second_cls, second_class=down_cls, nlayer=5, seed=seed, down_rate=down_rate)

### -Experiments: visualization

In [ ]:
seeds = [15213, 10086, 11777, 333, 444, 555, 666, 777, 888, 999]
datasets = ['DD', 'PROTEINS', 'NCI1', 'IMDB-BINARY', 'Mutagenicity', 'AIDS', 'ENZYMES-0-1', 'ENZYMES-2-3']#, 'COLLAB-0-1', 'COLLAB-1-2']
for name in datasets:
    d = name.split('-')
    data_name, down_cls, second_cls = d[0], 1, 0
    if len(d) == 2:
        data_name = d[0] + '-' + d[1] 
    if len(d) > 2:
        down_cls, second_cls = int(d[1]), int(d[2])
    investigate_model_embeddings(data_name, down_cls, second_cls)

# Combine all results into table

In [ ]:
import numpy as np
import pandas as pd 

files = list(filter(lambda x:('log' in x) and ('different' not in x), os.listdir('results')))
models = ['PK-LOF', 'PK-OCSVM', 'WL-LOF', 'WL-OCSVM', 'Graph2Vec-LOF', 'Graph2Vec-OCSVM', 'FGSD-LOF', 'FGSD-OCSVM', 'Graph2Vec-IF', 'FGSD-IF',  'OCGIN-5']
results = {model:{} for model in models} 

for file_name in files:
    file_name = os.path.join('results', file_name)
    with open(file_name, 'r') as file:
        lines = filter(lambda x:x[0]=='[' or x[0]=='!' or x[0]=='{', file.readlines())
        for line in lines:
            line = line.split()
            if len(line) == 5:
                line = [x.split('[') for x in line]
                line = {x[0]: x[1].strip(']') for x in line}
                key = line['!Data'] + '-' +  line['DownClass'] + '-' + line['SecondClass']
                model = line['Model']
                if not key in results[model]:
                    results[model][key] = []
            else:
                results[model][key].append(float(line[1].strip(',') ))   

import pandas as pd
# create performance table
rows = models
cols1 = ['DD-0-1', 'PROTEINS-0-1', 'NCI1-0-1', 'IMDB-BINARY-0-1', 'Mutagenicity-0-1', 'AIDS-0-1', 'ENZYMES-0-1', 'ENZYMES-2-3', 'COLLAB-0-1', 'COLLAB-1-2']
cols2 = ['DD-1-0', 'PROTEINS-1-0', 'NCI1-1-0', 'IMDB-BINARY-1-0', 'Mutagenicity-1-0', 'AIDS-1-0', 'ENZYMES-1-0', 'ENZYMES-3-2', 'COLLAB-1-0', 'COLLAB-2-1']
cols = [sub[item] for item in range(len(cols1))  for sub in [cols1, cols2]]
means = np.zeros((len(rows), len(cols)))
stds = np.zeros((len(rows), len(cols)))
for model, m_results in results.items():
    for data, result in m_results.items():
        result = np.array(result)
        if (model in rows) and (data in cols):
            means[rows.index(model), cols.index(data)] = round(result.mean(), 3) #result.mean()
            stds[rows.index(model), cols.index(data)] = round(result.std(), 3)
        else:
            print(f'Not in set: Model={model}, Data={data}, ROC-Mean={result.mean()}, STD={result.std()}') 

means = pd.DataFrame(means, index=rows, columns=cols)
stds =  pd.DataFrame(stds, index=rows, columns=cols)
means

# Take-aways and guidelines
In this paper we have observed several intriguing issues with repurposing binary graph classification datasets for graph-level outlier detection. Different from point-cloud based outlier detection where the pointwise representation is given, graph-level outlier detection requires some graph embedding methods to embed the unstructured and complicated graph data into pointwise representation and then conduct outlier detection. Based on this mechanism, there are several angles to stand for the root of observed issues: 1) inherent dataset problem; 2) issues of graph embedding methods; 3) issues of outlier detector. We have found that each angle raises different questions and future works and all angles are valid and reasonable. Instead of standing on a specific angle, we aim at summarizing arguments of every angle to inspire researchers. 

## Inherent dataset problem

## Is current graph embedding method "good" enough?

## Is the assumption of outlier detector suitable?
